# 케라스 임베딩 층

### 임베딩 층 사용하기

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**1) 문장의 긍, 부정을 판단하는 감성 분류 모델을 만들기 위해 긍정인 문장은 레이블 1, 부정인 문장은 레이블 0으로 지정**

In [ ]:
sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

**2) 케라스의 토크나이저를 사용하여 단어 집합을 만들고 크기를 확인**

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

16


**3) 각 문장에 대해서 정수 인코딩을 수행**

In [ ]:
X_encoded = tokenizer.texts_to_sequences(sentences)
print(X_encoded)

[[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]


**4) 가장 길이가 긴 문장의 길이를 구함**

In [ ]:
max_len = max(len(l) for l in X_encoded)
print(max_len)

4


**5) 최대 길이로 모든 샘플에 대해서 패딩을 진행**

In [ ]:
X_train = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train = np.array(y_train)
print(X_train)

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


**6) 출력층에 1개의 뉴런을 배치하고 활성화 함수로 시그모이드 함수, 손실 함수로 binary_crossentropy를 사용**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

embedding_dim = 4

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

**7) 100 에포크 학습하여 현재 각 단어들의 임베딩 벡터들의 값이 출력층의 가중치와 함께 학습됨**

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 2s - loss: 0.6845 - acc: 0.5714
Epoch 2/100
1/1 - 0s - loss: 0.6832 - acc: 0.5714
Epoch 3/100
1/1 - 0s - loss: 0.6818 - acc: 0.5714
Epoch 4/100
1/1 - 0s - loss: 0.6805 - acc: 0.7143
Epoch 5/100
1/1 - 0s - loss: 0.6791 - acc: 0.7143
Epoch 6/100
1/1 - 0s - loss: 0.6777 - acc: 0.7143
Epoch 7/100
1/1 - 0s - loss: 0.6764 - acc: 0.7143
Epoch 8/100
1/1 - 0s - loss: 0.6750 - acc: 0.8571
Epoch 9/100
1/1 - 0s - loss: 0.6736 - acc: 0.8571
Epoch 10/100
1/1 - 0s - loss: 0.6723 - acc: 0.8571
Epoch 11/100
1/1 - 0s - loss: 0.6709 - acc: 0.8571
Epoch 12/100
1/1 - 0s - loss: 0.6695 - acc: 0.8571
Epoch 13/100
1/1 - 0s - loss: 0.6682 - acc: 0.8571
Epoch 14/100
1/1 - 0s - loss: 0.6668 - acc: 0.8571
Epoch 15/100
1/1 - 0s - loss: 0.6654 - acc: 0.8571
Epoch 16/100
1/1 - 0s - loss: 0.6640 - acc: 0.8571
Epoch 17/100
1/1 - 0s - loss: 0.6626 - acc: 0.8571
Epoch 18/100
1/1 - 0s - loss: 0.6612 - acc: 0.8571
Epoch 19/100
1/1 - 0s - loss: 0.6598 - acc: 1.0000
Epoch 20/100
1/1 - 0s - loss: 0.6583 - a

# 사전 훈련된 워드 임베딩 사용하기

### 훈련 데이터 이해하고 전처리하기

**1) 앞서 케라스 임베딩 층에서 사용한 데이터를 사용**

### 사전 훈련된 GloVe 사용하기

In [ ]:
from urllib.request import urlretrieve, urlopen
import gzip
import zipfile

**1) glove.6B.zip을 다운로드하고 압축을 푼 후 glove.6B.100d.txt 파일을 사용**

In [ ]:
urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", filename="glove.6B.zip")
zf = zipfile.ZipFile('glove.6B.zip')
zf.extractall() 
zf.close()

**2) glove.6B.txt에 있는 모든 임베딩 벡터를 불러온 후 딕셔너리를 사용해 로드한 임베딩 벡터의 개수를 확인**

In [ ]:
embedding_dict = dict()

f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    
    # 100개의 값을 가지는 array로 변환
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32')
    embedding_dict[word] = word_vector_arr
f.close()

print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

400000개의 Embedding vector가 있습니다.


**3) 임의의 단어 'respectable'의 임베딩 벡터 값과 크기를 출력**

In [ ]:
print(embedding_dict['respectable'])
print(len(embedding_dict['respectable']))

[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0.17694
 -0.0090944  0.1109     0.

**4) 벡터의 차원 수가 100이므로 풀고자 하는 문제의 단어 집합 크기의 행과 100개의 열을 가지는 행렬을 생성하고 0으로 채운 후**<br>
&nbsp;&nbsp;&nbsp;&nbsp;   **이 후 이 행렬에 사전 훈련된 임베딩 값을 넣어주도록 할 것임**

In [ ]:
# 단어 집합 크기의 행과 100개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
embedding_matrix = np.zeros((vocab_size, 100))
np.shape(embedding_matrix)

(16, 100)

**5) 기존 데이터의 각 단어와 맵핑된 정수값을 확인하면 단어 'great'의 맵핑된 정수는 2**

In [ ]:
print(tokenizer.word_index.items())

dict_items([('nice', 1), ('great', 2), ('best', 3), ('amazing', 4), ('stop', 5), ('lies', 6), ('pitiful', 7), ('nerd', 8), ('excellent', 9), ('work', 10), ('supreme', 11), ('quality', 12), ('bad', 13), ('highly', 14), ('respectable', 15)])


In [ ]:
tokenizer.word_index['great']

2

**6) 사전 훈련된 GloVe에서 단어 'great'의 벡터값을 확인**

In [ ]:
print(embedding_dict['great'])

[-0.013786   0.38216    0.53236    0.15261   -0.29694   -0.20558
 -0.41846   -0.58437   -0.77355   -0.87866   -0.37858   -0.18516
 -0.128     -0.20584   -0.22925   -0.42599    0.3725     0.26077
 -1.0702     0.62916   -0.091469   0.70348   -0.4973    -0.77691
  0.66045    0.09465   -0.44893    0.018917   0.33146   -0.35022
 -0.35789    0.030313   0.22253   -0.23236   -0.19719   -0.0053125
 -0.25848    0.58081   -0.10705   -0.17845   -0.16206    0.087086
  0.63029   -0.76649    0.51619    0.14073    1.019     -0.43136
  0.46138   -0.43585   -0.47568    0.19226    0.36065    0.78987
  0.088945  -2.7814    -0.15366    0.01015    1.1798     0.15168
 -0.050112   1.2626    -0.77527    0.36031    0.95761   -0.11385
  0.28035   -0.02591    0.31246   -0.15424    0.3778    -0.13599
  0.2946    -0.31579    0.42943    0.086969   0.019169  -0.27242
 -0.31696    0.37327    0.61997    0.13889    0.17188    0.30363
 -1.2776     0.044423  -0.52736   -0.88536   -0.19428   -0.61947
 -0.10146   -0.26301  

**7) 단어 집합의 모든 단어에 대해서 사전 훈련된 GloVe의 임베딩 벡터들을 임베딩 행렬에 맵핑한 후** <br>
&nbsp;&nbsp;&nbsp;&nbsp;     **'great'의 벡터값이 의도한 인덱스의 위치에 삽입되었는지 확인하기 위해 인덱스 2에서의 값을 확인하면** <br>
&nbsp;&nbsp;&nbsp;&nbsp;      **이전에 확인한 사전에 훈련된 GloVe의 'great'의 벡터값과 일치하는 것을 볼 수 있음**

In [ ]:
for word, index in tokenizer.word_index.items():
    # 단어와 맵핑되는 사전 훈련된 임베딩 벡터값
    vector_value = embedding_dict.get(word)
    if vector_value is not None:
        embedding_matrix[index] = vector_value

In [ ]:
embedding_matrix[2]

array([-0.013786  ,  0.38216001,  0.53236002,  0.15261   , -0.29694   ,
       -0.20558   , -0.41846001, -0.58437002, -0.77354997, -0.87866002,
       -0.37858   , -0.18516   , -0.12800001, -0.20584001, -0.22925   ,
       -0.42598999,  0.3725    ,  0.26076999, -1.07019997,  0.62915999,
       -0.091469  ,  0.70348001, -0.4973    , -0.77691001,  0.66044998,
        0.09465   , -0.44893   ,  0.018917  ,  0.33146   , -0.35021999,
       -0.35789001,  0.030313  ,  0.22253001, -0.23236001, -0.19719   ,
       -0.0053125 , -0.25848001,  0.58081001, -0.10705   , -0.17845   ,
       -0.16205999,  0.087086  ,  0.63028997, -0.76648998,  0.51618999,
        0.14072999,  1.01900005, -0.43136001,  0.46138   , -0.43584999,
       -0.47567999,  0.19226   ,  0.36065   ,  0.78987002,  0.088945  ,
       -2.78139997, -0.15366   ,  0.01015   ,  1.17980003,  0.15167999,
       -0.050112  ,  1.26259995, -0.77526999,  0.36030999,  0.95761001,
       -0.11385   ,  0.28035   , -0.02591   ,  0.31246001, -0.15

**8) 임베딩 층에 embedding_matix를 초기값으로 설정하고 차원 수가 100이므로 output_dim을 100으로 설정함**<br>
&nbsp;&nbsp;&nbsp;&nbsp;     **그리고 사전 훈련된 워드 임베딩을 사용하므로 추가 훈련을 하지 않는다는 의미의 trainable의 인자값을 False로 선택해 훈련**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)

In [ ]:
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.7021 - acc: 0.5714
Epoch 2/100
1/1 - 0s - loss: 0.6829 - acc: 0.5714
Epoch 3/100
1/1 - 0s - loss: 0.6644 - acc: 0.5714
Epoch 4/100
1/1 - 0s - loss: 0.6466 - acc: 0.5714
Epoch 5/100
1/1 - 0s - loss: 0.6294 - acc: 0.5714
Epoch 6/100
1/1 - 0s - loss: 0.6129 - acc: 0.5714
Epoch 7/100
1/1 - 0s - loss: 0.5969 - acc: 0.7143
Epoch 8/100
1/1 - 0s - loss: 0.5816 - acc: 0.7143
Epoch 9/100
1/1 - 0s - loss: 0.5668 - acc: 0.7143
Epoch 10/100
1/1 - 0s - loss: 0.5525 - acc: 0.7143
Epoch 11/100
1/1 - 0s - loss: 0.5386 - acc: 0.7143
Epoch 12/100
1/1 - 0s - loss: 0.5253 - acc: 0.7143
Epoch 13/100
1/1 - 0s - loss: 0.5124 - acc: 0.7143
Epoch 14/100
1/1 - 0s - loss: 0.4999 - acc: 0.8571
Epoch 15/100
1/1 - 0s - loss: 0.4878 - acc: 0.8571
Epoch 16/100
1/1 - 0s - loss: 0.4760 - acc: 0.8571
Epoch 17/100
1/1 - 0s - loss: 0.4647 - acc: 0.8571
Epoch 18/100
1/1 - 0s - loss: 0.4536 - acc: 1.0000
Epoch 19/100
1/1 - 0s - loss: 0.4429 - acc: 1.0000
Epoch 20/100
1/1 - 0s - loss: 0.4325 - a

### 사전 훈련된 Word2Vec 사용하기

**1) 구글의 사전 훈련된 Word2Vec 모델을 gensim을 통해 로드하고 word2vec_model에 저장**

In [ ]:
import gensim

In [ ]:
urlretrieve("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", \
                           filename="GoogleNews-vectors-negative300.bin.gz")
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

**2) 모델의 크기를 확인하면 3000000 x 300이므로 3백만 개의 단어와 각 단어의 차원은 300**<br>
**3) 그렇기 때문에 3000000 x 300 x 4bytes를 하면 모델 파일의 크기는 3.3GB임을 알 수 있음**

In [ ]:
print(word2vec_model.vectors.shape) # 모델의 크기 확인

(3000000, 300)


**4) 벡터의 차원 수가 300이므로 풀고자 하는 문제의 단어 집합 크기의 행과 300개의 열을 가지는 행렬을 생성하고 0으로 채운 후**<br>
&nbsp;&nbsp;&nbsp;&nbsp;     **이 후 이 행렬에 사전 훈련된 임베딩 값을 넣어주도록 할 것임**

In [ ]:
# 단어 집합 크기의 행과 300개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
embedding_matrix = np.zeros((vocab_size, 300))
np.shape(embedding_matrix)

(16, 300)

**5) 특정 단어의 임베딩 벡터가 없으면 None을 리턴하는 함수 get_vector()를 구현한 후**<br>
&nbsp;&nbsp;&nbsp;&nbsp;     **이를 사용해 None이 아니라면 임베딩 행렬에 해당 단어의 인덱스 위치의 행에 임베딩 벡터의 값을 저장하여 맵핑하도록 함**

In [ ]:
def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

In [ ]:
for word, index in tokenizer.word_index.items():
    # 단어와 맵핑되는 사전 훈련된 임베딩 벡터값
    vector_value = get_vector(word)
    if vector_value is not None:
        embedding_matrix[index] = vector_value

**6) 사전 훈련된 Word2Vec에서 단어 'great'의 벡터값을 확인**

In [ ]:
print(word2vec_model['great'])

[ 7.17773438e-02  2.08007812e-01 -2.84423828e-02  1.78710938e-01
  1.32812500e-01 -9.96093750e-02  9.61914062e-02 -1.16699219e-01
 -8.54492188e-03  1.48437500e-01 -3.34472656e-02 -1.85546875e-01
  4.10156250e-02 -8.98437500e-02  2.17285156e-02  6.93359375e-02
  1.80664062e-01  2.22656250e-01 -1.00585938e-01 -6.93359375e-02
  1.04427338e-04  1.60156250e-01  4.07714844e-02  7.37304688e-02
  1.53320312e-01  6.78710938e-02 -1.03027344e-01  4.17480469e-02
  4.27246094e-02 -1.10351562e-01 -6.68945312e-02  4.19921875e-02
  2.50000000e-01  2.12890625e-01  1.59179688e-01  1.44653320e-02
 -4.88281250e-02  1.39770508e-02  3.55529785e-03  2.09960938e-01
  1.52343750e-01 -7.32421875e-02  2.16796875e-01 -5.76171875e-02
 -2.84423828e-02 -3.60107422e-03  1.52343750e-01 -2.63671875e-02
  2.13623047e-02 -1.51367188e-01  1.04003906e-01  3.18359375e-01
 -1.85546875e-01  3.68652344e-02 -1.10839844e-01 -3.17382812e-02
 -1.01562500e-01 -1.21093750e-01  3.22265625e-01 -7.32421875e-02
 -1.52343750e-01  2.67578

**7) 기존 데이터의 각 단어와 맵핑된 정수값을 확인하면 단어 'great'의 맵핑된 정수는 2**

In [ ]:
tokenizer.word_index['great']

2

**8) 'great의 벡터값이 의도한 인덱스의 위치에 삽입되었는지 확인하기 위해 인덱스 2에서의 값을 확인하면**<br>
&nbsp;&nbsp;&nbsp;&nbsp;      **이전에 확인한 사전에 훈련된 Word2Vec의 'great'의 벡터값과 일치하는 것을 볼 수 있음**

In [ ]:
print(embedding_matrix[2])

[ 7.17773438e-02  2.08007812e-01 -2.84423828e-02  1.78710938e-01
  1.32812500e-01 -9.96093750e-02  9.61914062e-02 -1.16699219e-01
 -8.54492188e-03  1.48437500e-01 -3.34472656e-02 -1.85546875e-01
  4.10156250e-02 -8.98437500e-02  2.17285156e-02  6.93359375e-02
  1.80664062e-01  2.22656250e-01 -1.00585938e-01 -6.93359375e-02
  1.04427338e-04  1.60156250e-01  4.07714844e-02  7.37304688e-02
  1.53320312e-01  6.78710938e-02 -1.03027344e-01  4.17480469e-02
  4.27246094e-02 -1.10351562e-01 -6.68945312e-02  4.19921875e-02
  2.50000000e-01  2.12890625e-01  1.59179688e-01  1.44653320e-02
 -4.88281250e-02  1.39770508e-02  3.55529785e-03  2.09960938e-01
  1.52343750e-01 -7.32421875e-02  2.16796875e-01 -5.76171875e-02
 -2.84423828e-02 -3.60107422e-03  1.52343750e-01 -2.63671875e-02
  2.13623047e-02 -1.51367188e-01  1.04003906e-01  3.18359375e-01
 -1.85546875e-01  3.68652344e-02 -1.10839844e-01 -3.17382812e-02
 -1.01562500e-01 -1.21093750e-01  3.22265625e-01 -7.32421875e-02
 -1.52343750e-01  2.67578

**9) 임베딩 층에 embedding_matix를 초기값으로 설정하고 차원 수가 300이므로 output_dim을 300으로 설정함**<br>
&nbsp;&nbsp;&nbsp;&nbsp;     **그리고 사전 훈련된 워드 임베딩을 사용하므로 추가 훈련을 하지 않는다는 의미의 trainable의 인자값을 False로 선택해 훈련**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Input

model = Sequential()
model.add(Input(shape=(max_len,), dtype='int32'))
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.7373 - acc: 0.4286
Epoch 2/100
1/1 - 0s - loss: 0.7183 - acc: 0.4286
Epoch 3/100
1/1 - 0s - loss: 0.6998 - acc: 0.7143
Epoch 4/100
1/1 - 0s - loss: 0.6819 - acc: 0.7143
Epoch 5/100
1/1 - 0s - loss: 0.6644 - acc: 0.7143
Epoch 6/100
1/1 - 0s - loss: 0.6475 - acc: 0.8571
Epoch 7/100
1/1 - 0s - loss: 0.6312 - acc: 0.8571
Epoch 8/100
1/1 - 0s - loss: 0.6153 - acc: 0.8571
Epoch 9/100
1/1 - 0s - loss: 0.5999 - acc: 0.8571
Epoch 10/100
1/1 - 0s - loss: 0.5850 - acc: 0.8571
Epoch 11/100
1/1 - 0s - loss: 0.5706 - acc: 0.8571
Epoch 12/100
1/1 - 0s - loss: 0.5566 - acc: 0.8571
Epoch 13/100
1/1 - 0s - loss: 0.5431 - acc: 0.8571
Epoch 14/100
1/1 - 0s - loss: 0.5300 - acc: 1.0000
Epoch 15/100
1/1 - 0s - loss: 0.5174 - acc: 1.0000
Epoch 16/100
1/1 - 0s - loss: 0.5051 - acc: 1.0000
Epoch 17/100
1/1 - 0s - loss: 0.4932 - acc: 1.0000
Epoch 18/100
1/1 - 0s - loss: 0.4817 - acc: 1.0000
Epoch 19/100
1/1 - 0s - loss: 0.4705 - acc: 1.0000
Epoch 20/100
1/1 - 0s - loss: 0.4597 - a